In [ ]:
pip install neo4j


   ---------------------------------------- 0.0/312.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/312.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/312.3 kB ? eta -:--:--
   ------- ------------------------------- 61.4/312.3 kB 656.4 kB/s eta 0:00:01
   ------------------------------ --------- 235.5/312.3 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 312.3/312.3 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import  neo4j
import pandas as pd
import pyodbc

In [3]:


server = 'mcruebs04.isad.isadroot.ex.ac.uk'
database = 'BEMM459_GroupO'
username = 'GroupO'
password = 'MrjV827*Wr'

try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};',
        autocommit=True
    )
    print("Connected to Microsoft SQL Server!")
except Exception as e:
    print("Error connecting to SQL Server:", e)


Connected to Microsoft SQL Server!


In [3]:
try:
# Create cursor
    cursor = conn.cursor()
    
    # List tables to drop
    tables_to_drop = [
        "Tracking_Info",
        "Shipments",
        "Delivery_Locations",
        "Dispatch_Locations",
        "Parcels",
        "Customers"
    ]
    
    # Drop each table
    for table in tables_to_drop:
        try:
            # Simple drop table statement
            cursor.execute(f"DROP TABLE IF EXISTS {table}")
            print(f"Table {table} dropped successfully!")
        except Exception as e:
            print(f"Error dropping table {table}: {e}")
    
    # Commit changes
    conn.commit()
    print("All changes committed")
    
    # Close resources
    cursor.close()
    conn.close()
    print("All tables deleted successfully!")
    
except Exception as e:
    print(f"Error: {e}")

Error: name 'conn' is not defined


In [4]:
cursor = conn.cursor()



# Create tables
tables = [
"""  
CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(100),
    Email VARCHAR(100),
    Phone VARCHAR(20),
    Address VARCHAR(200),
    Type VARCHAR(50))
"""

"""
CREATE TABLE Parcels (
    ParcelID INT PRIMARY KEY,
    ParcelName VARCHAR(100),
    CustomerID INT,
    Weight FLOAT,
    Type VARCHAR(50),
    ShippingMethod VARCHAR(50),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
)"""

 """
CREATE TABLE Shipments (
    ShipmentID INT PRIMARY KEY,
    ShipmentName VARCHAR(100),
    ParcelID INT,
    ShipmentDate DATETIME,
    DeliveryDate DATETIME,
    Status VARCHAR(50),
    CurrentLocation VARCHAR(100),
    FOREIGN KEY (ParcelID) REFERENCES Parcels(ParcelID)
)"""

"""
CREATE TABLE Personnel (
    PersonnelID INT PRIMARY KEY,
    Name VARCHAR(100),
    Phone VARCHAR(20),
    Role VARCHAR(50)
)"""

"""
CREATE TABLE ShipmentPersonnel (
    ShipmentID INT,
    PersonnelID INT,
    FOREIGN KEY (ShipmentID) REFERENCES Shipments(ShipmentID),
    FOREIGN KEY (PersonnelID) REFERENCES Personnel(PersonnelID)
);"""

"""
CREATE TABLE Analytics (
    ShipmentID INT,
    Delays INT,
    Efficiency FLOAT,
    CustomerRating INT,
    FOREIGN KEY (ShipmentID) REFERENCES Shipments(ShipmentID)
)"""
        
]

for table in tables:
    try:
        cursor.execute(table)
        print("Table created successfully!")
    except Exception as e:
        print("Error creating table:", e)

conn.commit()
cursor.close()
conn.close()


NameError: name 'conn' is not defined

In [5]:
from neo4j import GraphDatabase

# Neo4j connection details
URI = "bolt://localhost:7687"  # Update if using a different URI
USERNAME = "neo4j"              # Default username
PASSWORD = "12345678"           # Update with your actual password

# Connect to the Neo4j database
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

def create_node(tx, label, **properties):
    """
    Creates a node with the given label and properties.
    """
    query = f"CREATE (n:{label} {{{', '.join(f'{key}: ${key}' for key in properties.keys())}}})"
    tx.run(query, **properties)

def create_relationship(tx, label1, prop1, label2, prop2, relationship):
    """
    Creates a relationship between two nodes.
    """
    query = f"""
    MATCH (a:{label1} {{ID: $id1}}), (b:{label2} {{ID: $id2}})
    CREATE (a)-[:{relationship}]->(b)
    """
    tx.run(query, id1=prop1['ID'], id2=prop2['ID'])

# Main function to set up the database structure
def setup_database():
    with driver.session() as session:
        # Create nodes
        session.write_transaction(create_node, "Customer", ID="C1", Name="John Doe", Type="Individual", ContactInfo="john@example.com")
        session.write_transaction(create_node, "Parcel", ID="P1", Name="Electronics Box", Weight="5kg", Type="Fragile")
        session.write_transaction(create_node, "Shipment", ID="S1", Status="In Transit", Dates="2025-03-30", Location="Warehouse A")
        session.write_transaction(create_node, "Personnel", ID="PR1", Name="Alice", Role="Driver")
        session.write_transaction(create_node, "Analytics", ID="A1", Metrics="Delivery Time", Insights="On Time")

        # Create relationships
        session.write_transaction(create_relationship, "Customer", {"ID": "C1"}, "Parcel", {"ID": "P1"}, "OWNS")
        session.write_transaction(create_relationship, "Parcel", {"ID": "P1"}, "Shipment", {"ID": "S1"}, "SHIPPED_IN")
        session.write_transaction(create_relationship, "Shipment", {"ID": "S1"}, "Personnel", {"ID": "PR1"}, "HANDLED_BY")
        session.write_transaction(create_relationship, "Shipment", {"ID": "S1"}, "Analytics", {"ID": "A1"}, "HAS_ANALYTICS")

    print("Database structure set up successfully!")

# Run the setup
setup_database()

# Close the connection
driver.close()


C:\Users\bgane\AppData\Local\Temp\ipykernel_4580\1787676351.py:32: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Customer", ID="C1", Name="John Doe", Type="Individual", ContactInfo="john@example.com")
C:\Users\bgane\AppData\Local\Temp\ipykernel_4580\1787676351.py:33: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Parcel", ID="P1", Name="Electronics Box", Weight="5kg", Type="Fragile")
C:\Users\bgane\AppData\Local\Temp\ipykernel_4580\1787676351.py:34: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Shipment", ID="S1", Status="In Transit", Dates="2025-03-30", Location="Warehouse A")
C:\Users\bgane\AppData\Local\Temp\ipykernel_4580\1787676351.py:35: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Personnel", ID="PR1", Name="Ali

Database structure set up successfully!


In [2]:
import pandas as pd
import pyodbc

# SQL Data Loader
class SQLDataLoader:
    def __init__(self, db_url):
        self.conn = pyodbc.connect(db_url)

    def load_data(self, file_path, sheet_name, table_name):
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        cursor = self.conn.cursor()
        for _, row in df.iterrows():
            placeholders = ', '.join(['?' for _ in row])
            query = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(query, tuple(row))
        self.conn.commit()
        cursor.close()
        print(f"Data loaded into {table_name} successfully!")

# Usage Example
sql_loader = SQLDataLoader("DRIVER={ODBC Driver 17 for SQL Server};SERVER=mcruebs04.isad.isadroot.ex.ac.uk;DATABASE=BEMM459_GroupO;UID=GroupO;PWD=MrjV827*Wr")

# Load data into each table
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Customers", "Customers")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Parcels", "Parcels")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Shipments", "Shipments")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Personnel", "Personnel")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "ShipmentPersonnel", "ShipmentPersonnel")
sql_loader.load_data(r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx", "Analytics", "Analytics")


Data loaded into Customers successfully!
Data loaded into Parcels successfully!
Data loaded into Shipments successfully!
Data loaded into Personnel successfully!
Data loaded into ShipmentPersonnel successfully!
Data loaded into Analytics successfully!


In [6]:

cursor = sql_loader.conn.cursor()

tables = ["Customers", "Parcels", "Shipments", "Personnel", "ShipmentPersonnel", "Analytics"]

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"Table {table} has {count} rows.")




for table in tables:
    cursor.execute(f"SELECT TOP 5 * FROM {table}")
    rows = cursor.fetchall()
    print(f"\nPreview of {table}:")
    for row in rows:
        print(row)
cursor.close()


Table Customers has 83 rows.
Table Parcels has 269 rows.
Table Shipments has 269 rows.
Table Personnel has 27 rows.
Table ShipmentPersonnel has 556 rows.
Table Analytics has 269 rows.

Preview of Customers:
(1, 'Customer_1', 'customer1@example.com', '+44 7126855092', 'Address_1', 'Premium')
(2, 'Customer_2', 'customer2@example.com', '+44 7362950628', 'Address_2', 'Regular')
(3, 'Customer_3', 'customer3@example.com', '+44 7249827706', 'Address_3', 'Regular')
(4, 'Customer_4', 'customer4@example.com', '+44 7826600539', 'Address_4', 'Regular')
(5, 'Customer_5', 'customer5@example.com', '+44 7734036506', 'Address_5', 'Premium')

Preview of Parcels:
(1, 'Parcel_1', 68, 18.41062314071619, 'Perishable', 'Express')
(2, 'Parcel_2', 3, 2.681420643757454, 'Perishable', 'Express')
(3, 'Parcel_3', 31, 1.629540746636657, 'Fragile', 'Standard')
(4, 'Parcel_4', 63, 16.41245344169172, 'Solid', 'Standard')
(5, 'Parcel_5', 61, 18.96354925424732, 'Solid', 'Express')

Preview of Shipments:
(1, 'Shipment_1'

In [7]:

cursor = sql_loader.conn.cursor()
#Function to get column names and data types
def get_column_info(table_name):
    query = f"""
    SELECT COLUMN_NAME, DATA_TYPE
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = '{table_name}';
    """
    cursor.execute(query)
    print(f"\nColumns for table '{table_name}':")
    for row in cursor.fetchall():
        print(f"Column Name: {row.COLUMN_NAME}, Data Type: {row.DATA_TYPE}")

# Loop through each table and print the column details
for table in tables:
    get_column_info(table)

# Close connection
cursor.close()
conn.close()


Columns for table 'Customers':
Column Name: CustomerID, Data Type: int
Column Name: Name, Data Type: varchar
Column Name: Email, Data Type: varchar
Column Name: Phone, Data Type: varchar
Column Name: Address, Data Type: varchar
Column Name: Type, Data Type: varchar

Columns for table 'Parcels':
Column Name: ParcelID, Data Type: int
Column Name: ParcelName, Data Type: varchar
Column Name: CustomerID, Data Type: int
Column Name: Weight, Data Type: float
Column Name: Type, Data Type: varchar
Column Name: ShippingMethod, Data Type: varchar

Columns for table 'Shipments':
Column Name: ShipmentID, Data Type: int
Column Name: ShipmentName, Data Type: varchar
Column Name: ParcelID, Data Type: int
Column Name: ShipmentDate, Data Type: datetime
Column Name: DeliveryDate, Data Type: datetime
Column Name: Status, Data Type: varchar
Column Name: CurrentLocation, Data Type: varchar

Columns for table 'Personnel':
Column Name: PersonnelID, Data Type: int
Column Name: Name, Data Type: varchar
Column 

In [9]:
from neo4j import GraphDatabase
import pandas as pd

# Neo4j connection details
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "12345678"

# Connect to the Neo4j database
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# Load Excel data
file_path = r"C:\Users\bgane\Downloads\dhl\logistics_data.xlsx"
excel_data = pd.ExcelFile(file_path)

# Define functions to create nodes and relationships
def create_node(tx, label, **properties):
    query = f"CREATE (n:{label} {{{', '.join(f'{key}: ${key}' for key in properties.keys())}}})"
    tx.run(query, **properties)

def create_relationship(tx, label1, prop1, label2, prop2, relationship):
    query = f"""
    MATCH (a:{label1} {{ID: $id1}}), (b:{label2} {{ID: $id2}})
    CREATE (a)-[:{relationship}]->(b)
    """
    tx.run(query, id1=prop1['ID'], id2=prop2['ID'])

# Load data into Neo4j
def load_data():
    with driver.session() as session:
        # Load and create Customer nodes
        customers_df = excel_data.parse('Customers')
        for _, row in customers_df.iterrows():
            session.write_transaction(create_node, "Customer", ID=str(row['CustomerID']), Name=row['Name'], Type=row['Type'], Email=row['Email'])

        # Load and create Parcel nodes and relationships
        parcels_df = excel_data.parse('Parcels')
        for _, row in parcels_df.iterrows():
            session.write_transaction(create_node, "Parcel", ID=str(row['ParcelID']), Name=row['ParcelName'], Weight=str(row['Weight']), Type=row['Type'])
            session.write_transaction(create_relationship, "Customer", {"ID": str(row['CustomerID'])}, "Parcel", {"ID": str(row['ParcelID'])}, "OWNS")

        # Load and create Shipment nodes
        shipments_df = excel_data.parse('Shipments')
        for _, row in shipments_df.iterrows():
            session.write_transaction(create_node, "Shipment", ID=str(row['ShipmentID']), Status=row['Status'], Dates=row['Dates'], Location=row['Location'])
            session.write_transaction(create_relationship, "Parcel", {"ID": str(row['ParcelID'])}, "Shipment", {"ID": str(row['ShipmentID'])}, "SHIPPED_IN")

        # Load and create Personnel nodes
        personnel_df = excel_data.parse('Personnel')
        for _, row in personnel_df.iterrows():
            session.write_transaction(create_node, "Personnel", ID=str(row['PersonnelID']), Name=row['Name'], Role=row['Role'])

        # Create Shipment-Personnel relationships
        shipment_personnel_df = excel_data.parse('ShipmentPersonnel')
        for _, row in shipment_personnel_df.iterrows():
            session.write_transaction(create_relationship, "Shipment", {"ID": str(row['ShipmentID'])}, "Personnel", {"ID": str(row['PersonnelID'])}, "HANDLED_BY")

        # Load and create Analytics nodes and relationships
        analytics_df = excel_data.parse('Analytics')
        for _, row in analytics_df.iterrows():
            session.write_transaction(create_node, "Analytics", ID=str(row['ShipmentID']), Metrics=row['Metrics'], Insights=row['Insights'])
            session.write_transaction(create_relationship, "Shipment", {"ID": str(row['ShipmentID'])}, "Analytics", {"ID": str(row['ShipmentID'])}, "HAS_ANALYTICS")

    print("Data loaded successfully!")

# Run the data loading function
load_data()

driver.close()


C:\Users\bgane\AppData\Local\Temp\ipykernel_25064\1478771781.py:34: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, "Customer", ID=str(row['CustomerID']), Name=row['Name'], Type=row['Type'], Email=row['Email'])
Transaction failed and will be retried in 1.1548458305421132s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No connection could be made because the target machine actively refused it))
Transaction failed and will be retried in 2.0371911291660973s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (rea

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [WinError 10061] No connection could be made because the target machine actively refused it)